In [36]:
import toolz as tz
import operator as op


In [3]:
class Container:
    def __init__(self, x):
        self.__value__ = x

    @classmethod
    def of(cls, x):
        return cls(x)

    def map(self, f):
        return Container.of(f(self.__value__))

    def __repr__(self):
        return str(self.__value__)


class Maybe:
    def __init__(self, x):
        self.__value__ = x
        self.isNothing = self.__value__ is None

    @classmethod
    def of(cls, x):
        return cls(x)

    def map(self, f):
        return self if self.isNothing else Maybe.of(f(self.__value__))

    def __repr__(self):
        return "Nothing" if self.isNothing else f"Just({self.__value__})"


@tz.curry
def prop(name, obj):
    return obj[name]


@tz.curry
def map_(f, functor):
    return functor.map(f)


@tz.curry
def maybe(v, f, m):
    return v if m.isNothing else f(m.__value__)


In [4]:
Container.of(3).map(lambda two: two + 2)


5

In [5]:
Maybe.of("abc").map(len), Maybe.of("").map(len)


(Just(3), Just(0))

In [6]:
map_(len, Container.of("abc"))


3

In [7]:
@tz.curry
def withdraw(amount, balance):
    return Maybe.of({"balance": balance - amount} if balance >= amount else None)


def updateLedger(account):
    return account


def remainingBalance(balance):
    return f"Your balance is ${balance}"


finishTransaction = tz.compose(remainingBalance, updateLedger)

getTwenty = tz.compose(
    maybe(f"You're broke!", finishTransaction), withdraw(20), prop("balance")
)

getTwenty({"balance": 100}), getTwenty({"balance": 10})


("Your balance is ${'balance': 80}", "You're broke!")

In [34]:
class Either:
    def __init__(self, x):
        self.__value__ = x

    @classmethod
    def of(cls, x):
        return Right(x)


class Right(Either):
    def map(self, f):
        return Either.of(f(self.__value__))

    def __repr__(self):
        return f"Right({self.__value__})"


class Left(Either):
    def map(self, _):
        return self

    def __repr__(self):
        return f"Left({self.__value__})"


left = lambda x: Left(x)


@tz.curry
def either(f, g, e):
    if isinstance(e, Left):
        return f(e.__value__)
    elif isinstance(e, Right):
        return g(e.__value__)

    return None


In [15]:
(
    Either.of("rain").map(lambda str: f"b{str}"),
    left("rain").map(lambda str: f"It's gonna {str}, better bring your umbrella!"),
    Either.of({"host": "localhost", "port": 80}).map(prop("host")),
    left("rolls eyes...").map(prop("host")),
)


(Right(brain), Left(rain), Right(localhost), Left(rolls eyes...))

In [43]:
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from datetime import datetime

@tz.curry
def getAge(now, user):
    try:
        birthDate = parse(user["birthDate"])

        time_diff = relativedelta(now, birthDate)

        return Either.of(time_diff.years)
    except:
        return Left(f"Birth date could not be parsed")


(
    getAge(datetime.now(), {"birthDate": "2020-12-12"}),
    getAge(datetime.now(), {"birthDate": "2005-13-12"}),
)


(Right(1), Left(Birth date could not be parsed))

In [44]:
concat = tz.curry(op.add)
add = concat

fortune = tz.compose(
    concat('If you survive, you will be '),
    str,
    add(1)
)

zol = tz.compose(
    map_(print),
    map_(fortune),
    getAge(datetime.now())
)

zol({"birthDate": '2012-5-25'}), zol({"birthDate": '2012-15-25'})

If you survive, you will be 11


(Right(None), Left(Birth date could not be parsed))

In [33]:
z = tz.compose(
    print,
    either()
)

(True, True, False)